# Data Wrangling the Diamonds dataset

## A quick note --

There are two ways to run this notebook, depending on your familiarly with cleaning data.

Option 1 "I been doing this for a while chief" -- Run lines 1 & 2 and you're good to go. The pickle file is used for the EDA notebook.

Option 2 "I'm new to data munging so let's step though this". -- Skip line 2 

## The basics

In [1]:
import pandas as pd

In [2]:
diamonds = (
    pd.read_csv("../data/diamonds_raw.csv", index_col=[0])
    .drop(['date', 'diamond_id'], axis=1)
    .rename(columns={'cut': 'cut_quality', 'shape' : 'cut'})
    .fillna({'color':'unknown',
             'cut_quality':'unknown', 
             'eye_clean': 'unknown',
             'fancy_color_dominant_color': 'unknown', 
             'fancy_color_secondary_color': 'unknown',
             'fancy_color_overtone':'unknown', 
             'fancy_color_intensity':'unknown',  
             'girdle_min':'unknown', 
             'girdle_max':'unknown', 
             'culet_size':'unknown', 
             'culet_condition':'unknown',
             'fluor_color':'unknown', 
             'fluor_intensity':'unknown' })
    .to_pickle("../data/diamonds.pkl"))

In [3]:
df = pd.read_csv("../data/diamonds_raw.csv", index_col=[0])

In [4]:
df.tail(3)

,diamond_id,shape,size,color,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,clarity,cut,...,girdle_min,girdle_max,culet_size,culet_condition,fluor_color,fluor_intensity,lab,total_sales_price,eye_clean,date
219701,135553116,Round,18.07,E,NaN,NaN,NaN,NaN,VS1,Excellent,...,TN,M,N,NaN,NaN,None,GIA,1315496,NaN,2022-02-24
219702,114752541,Princess,0.90,NaN,Red,NaN,NaN,Fancy,SI2,NaN,...,XTN,VTK,N,NaN,NaN,Faint,GIA,1350000,NaN,2022-02-24
219703,129630500,Pear,10.03,NaN,Yellow,NaN,NaN,Fancy Vivid,VVS2,NaN,...,NaN,NaN,NaN,NaN,NaN,None,GIA,1449881,NaN,2022-02-24


#### Let's start with a quick inspection of the dataframe

In [5]:
df.shape

(219704, 27)

In [6]:
df.info

<bound method DataFrame.info of         diamond_id     shape   size color fancy_color_dominant_color  \
0        131328926     Round   0.09     E                        NaN   
1        131704776     Round   0.09     E                        NaN   
2        131584417     Round   0.09     E                        NaN   
3        131584416     Round   0.09     E                        NaN   
4        131613644     Round   0.09     E                        NaN   
...            ...       ...    ...   ...                        ...   
219699   135619262     Round  10.65     E                        NaN   
219700   135122237   Radiant   5.17   NaN                      Green   
219701   135553116     Round  18.07     E                        NaN   
219702   114752541  Princess   0.90   NaN                        Red   
219703   129630500      Pear  10.03   NaN                     Yellow   

       fancy_color_secondary_color fancy_color_overtone fancy_color_intensity  \
0                     

## Identify and drop useless columns

#### Are the dates all the same?

In [7]:
df['date'].unique()

array(['2022-02-24'], dtype=object)

Well, that's a useless column. `diamond_id` could make a decent index, but a serialized count is more useful in this case because we're looking to develop a pricing model.
 
So, let's ditch them.

In [8]:
df = df.drop(['date', 'diamond_id'], axis=1)
df.head(2)

,shape,size,color,fancy_color_dominant_color,fancy_color_secondary_color,fancy_color_overtone,fancy_color_intensity,clarity,cut,symmetry,...,meas_depth,girdle_min,girdle_max,culet_size,culet_condition,fluor_color,fluor_intensity,lab,total_sales_price,eye_clean
0,Round,0.09,E,NaN,NaN,NaN,NaN,VVS2,Excellent,Very Good,...,1.79,M,M,N,NaN,NaN,None,IGI,200,NaN
1,Round,0.09,E,NaN,NaN,NaN,NaN,VVS2,Very Good,Very Good,...,1.78,STK,STK,N,NaN,NaN,None,IGI,200,NaN


#### What are the possible answers for the categorical variables?


In [9]:
col_list = ['color', 'clarity', 'cut', 'symmetry','polish','lab','eye_clean', 'culet_size', 'lab','shape',
            'fancy_color_intensity','fancy_color_dominant_color','fancy_color_secondary_color',
            'fancy_color_overtone', 'fluor_color', 'fluor_intensity',]
            
for col in col_list:
    print(f" '{col}' has the following values: \n \t {df[col].unique()} \n")

 'color' has the following values: 
 	 ['E' 'F' 'L' 'D' 'J' 'I' 'G' 'H' 'M' 'K' nan] 

 'clarity' has the following values: 
 	 ['VVS2' 'VVS1' 'I1' 'VS1' 'VS2' 'IF' 'SI2' 'I2' 'SI1' 'SI3' 'I3'] 

 'cut' has the following values: 
 	 ['Excellent' 'Very Good' nan 'Good' 'Fair' 'None' 'Ideal'] 

 'symmetry' has the following values: 
 	 ['Very Good' 'Excellent' 'Good' 'Fair' 'Poor'] 

 'polish' has the following values: 
 	 ['Very Good' 'Excellent' 'Good' 'Fair' 'Poor'] 

 'lab' has the following values: 
 	 ['IGI' 'GIA' 'HRD'] 

 'eye_clean' has the following values: 
 	 [nan 'Yes' 'E1' 'Borderline' 'No'] 

 'culet_size' has the following values: 
 	 ['N' nan 'S' 'M' 'VS' 'L' 'EL' 'SL' 'VL'] 

 'lab' has the following values: 
 	 ['IGI' 'GIA' 'HRD'] 

 'shape' has the following values: 
 	 ['Round' 'Pear' 'Oval' 'Marquise' 'Princess' 'Emerald' 'Heart' 'Cushion'
 'Radiant' 'Cushion Modified' 'Asscher'] 

 'fancy_color_intensity' has the following values: 
 	 [nan 'Fancy' 'Very Light' 'Fai

This is a little odd but something you don't really see until the EDA notebook. The diamond industry talks about the 4Cs: cut, clarity, color and caret weight. But here, cut refers to the quality of the cut, and they're using shape to mean cut. We're going to want to rename that to avoid confusion.

In [10]:
df = df.rename(columns={'cut': 'cut_quality', 'shape' : 'cut'})


#### Also, that seemed like a lot of NaNs in cell 6. But how many is that for each column?

In [11]:
for col in df:
    print(f"There are {df[col].isnull().sum()} nans in {col}")

There are 0 nans in cut
There are 0 nans in size
There are 9162 nans in color
There are 210540 nans in fancy_color_dominant_color
There are 218642 nans in fancy_color_secondary_color
There are 217666 nans in fancy_color_overtone
There are 210542 nans in fancy_color_intensity
There are 0 nans in clarity
There are 60607 nans in cut_quality
There are 0 nans in symmetry
There are 0 nans in polish
There are 0 nans in depth_percent
There are 0 nans in table_percent
There are 0 nans in meas_length
There are 0 nans in meas_width
There are 0 nans in meas_depth
There are 83433 nans in girdle_min
There are 84296 nans in girdle_max
There are 85741 nans in culet_size
There are 204385 nans in culet_condition
There are 203978 nans in fluor_color
There are 128 nans in fluor_intensity
There are 0 nans in lab
There are 0 nans in total_sales_price
There are 156917 nans in eye_clean


## Dealing with NaN values

#### Before I commit to just dropping all the NaNs, let's get an idea of how what that would look like.


```
df.shape #(219704, 25)
df.dropna(axis=1, inplace=True)
df.shape #(219704, 12)
```
That would result in a table that looks like this:

| diamond_id | cut | size | clarity | symmetry | polish  | depth_percent | table_percent | meas_length | meas_width | meas_depth | lab | total_sales_price|
|---|---|---|---|---|---|---|---|---|---|---|---|---|			
| 131328926 | Round | 0.09 | VVS2 | Very Good | Very Good | 62.7 | 59.0 | 2.85 | 2.87 | 1.79 | IGI | 200 |
| 131704776 | Round | 0.09 | VVS2 | Very Good | Very Good | 61.9 | 59.0 | 2.84 | 2.89 | 1.78 | IGI | 200 |
| 131584417 | Round | 0.09 | VVS2 | Very Good | Very Good | 61.1 | 59.0 | 2.88 | 2.90 | 1.77 | IGI | 200 |


And check of `df.isnull().sum().sum()` would return 0 (no null values), as we would expect.


#### Dropping the `NaN` values would keep all the rows (which is good) but we'd loose columns (which could be bad)


This wouldn't be bad in that it doesn't get rid of too many columns but the ones it does are important. Carat, Color, Cut, and Clarity are what the diamond industry are the factors.

But let's think about this. Say certain colored diamonds are really valuable. If we got rid of the columns, there would be nothing to analyze.

So, this needs to be fixed.

Now my sense is were going to find that isn't a big deal and also that girdle and cutlet are going to end up being functions of caret weight but, we'll find out later.

In [12]:
df = df.fillna({'color':'unknown',
             'cut_quality':'unknown', 
             'eye_clean': 'unknown',
             'fancy_color_dominant_color': 'unknown', 
             'fancy_color_secondary_color': 'unknown',
             'fancy_color_overtone':'unknown', 
             'fancy_color_intensity':'unknown',  
             'girdle_min':'unknown', 
             'girdle_max':'unknown', 
             'culet_size':'unknown', 
             'culet_condition':'unknown',
             'fluor_color':'unknown', 
             'fluor_intensity':'unknown' })

And a quick check to make sure there are now no nulls in this dataset.

In [13]:
df.isnull().sum().sum()

0

OK, from here we can package it up. We could save this a csv but I'm going to save it as a pickel file because it's very fast AND because we're using it internally. Pickle files aren't secure as they allow you run whatever is there. But in this case, we see all the steps we've done it's ok.

In [14]:
df.to_pickle("../data/diamonds.pkl")